In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tqdm import tqdm

In [3]:
import malaya

In [4]:
# model = malaya.dependency.transformer(model = 'xlnet', quantized = True)

In [5]:
# https://ms.wikipedia.org/wiki/Penjodoh_bilangan_bahasa_Melayu
penjodoh_bilangan = [
    'angkatan',
    'baris',
    'batang',
    'bentuk',
    'bidang',
    'biji',
    'bilah',
    'buah',
    'buku',
    'bungkus',
    'butir',
    'carik',
    'cebis',
    'cekak',
    'cubit',
    'cucuk',
    'das',
    'deret',
    'ekor',
    'gugus',
    'gelung',
    'gemal',
    'genggam',
    'gulung',
    'gumpal',
    'helai',
    'hidangan',
    'hiris',
    'ikat',
    'jambak',
    'jambangan',
    'jemput',
    'kaki',
    'kalung',
    'kandang',
    'kapur',
    'kawan',
    'kelompok',
    'kepal',
    'keping',
    'kepul',
    'kerat',
    'ketul',
    'kotak',
    'kuntum',
    'laras',
    'lembar',
    'lingkar',
    'longgok',
    'naskhah',
    'orang',
    'papan',
    'pasang',
    'pasukan',
    'patah',
    'pintu',
    'potong',
    'pucuk',
    'puntung',
    'rangkap',
    'rawan',
    'ruas',
    'rumpun',
    'sikat',
    'sisir',
    'suap',
    'tandan',
    'tangkai',
    'teguk',
    'timbun',
    'titik',
    'tongkol',
    'ulas',
    'untai',
    'urat',
    'utas',
]
hubung_list = [
    'agar',
    'apabila',
    'atau',
    'bahawa',
    'dan',
    'hingga',
    'jika',
    'jikalau',
    'kecuali',
    'kerana',
    'lalu',
    'manakala',
    'sambil',
    'serta',
    'semenjak',
    'sementara',
    'sungguhpun',
    'supaya',
    'walaupun',
    'tetapi',
    'berkenan',
    'berkenaan',
    'yang',
    'juga',
    'tersebut'
]
end = ['nya']
reserved = ['mereka', 'pelajar', 'rakyat', 'penduduk', 'umat', 'kami', 'semua', 'kumpulan', 'para']
start = ['be', 'ber', 'ter', 'se']

In [6]:
sepenjodoh_bilangan = [f'se{w}' for w in penjodoh_bilangan]
set_sepenjodoh_bilangan = set(sepenjodoh_bilangan)
set_penjodoh_bilangan = set(penjodoh_bilangan)
set_reserved = set(reserved)

In [7]:
with open('filtered-dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]

len(data)

2037249

In [8]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize
sastrawi = malaya.stem.sastrawi()

In [9]:
# graph, tagging, indexing = model.predict(' '.join(tokenizer('ibu bapa kita bekerja penat lelah')))
# graph.to_graphvis()

In [10]:
import copy
import re
from malaya.text.regex import _expressions

def reset_t(tokens):
    t = []
    for i in range(len(tokens)):
        t.append([tokens[i], 2])
    return t

# [penjodoh bilangan] [kata nama] -> [penjodoh bilangan] [kata nama - kata nama]
# dua buah kereta -> dua buah kereta-kereta
def augment_4_0(t, row):
    text, tokens, tokens_lower, penjodoh = row
    for word in penjodoh:
        try:
            i = tokens_lower.index(word) + 1
            if tokens_lower[i] in hubung_list:
                continue
            if tokens[i][0].isupper():
                continue
            if tokens[i].endswith('nya'):
                tokens[i] = tokens[i][:-3]
                ends = 'nya'
            else:
                ends = ''
            t[i][0] = f'{tokens[i]}-{tokens[i]}{ends}'
            t[i][1] = 4
        except Exception as e:
            print(e)
            pass

# [kata nama - kata nama] -> [kata nama]
# ayam-ayam itu -> ayam itu
def augment_4_1(t, row):
    text, tokens, tokens_lower, penjodoh = row
    for no, word in enumerate(tokens):
        if re.findall(_expressions['hypen'], word.lower()):
            stemmed = sastrawi.stem(word)
            if stemmed != word.split('-')[0]:
                continue
            if word[0].isupper():
                continue
            word = word.split('-')[0]
            t[no][0] = word
            t[no][1] = 4
            
def augment_4_2(t, row):
    text, tokens, tokens_lower, penjodoh = row
    for word in penjodoh:
        try:
            i = tokens_lower.index(word)
            if tokens[i].endswith('nya'):
                tokens[i] = tokens[i][:-3]
                ends = 'nya'
            else:
                ends = ''
            t[i][0] = f'{tokens[i]}-{tokens[i]}{ends}'
            t[i][1] = 4
        except Exception as e:
            print(e)
            pass
    
            
# [kata majmuk] -> [kata majmuk - kata majmuk]
# ibu-ibu bapa -> ibu bapa-ibu bapa
# def augment_4_2(t, row):
    

In [13]:
results = []
for text in tqdm(['berjuta-juta itu']):
    tokens = tokenizer(text)
    t = reset_t(tokens)
    t_ = copy.deepcopy(t)
    tokens_lower = tokenizer(text.lower())
    set_tokens = set(tokens_lower)
    r_penjodoh_bilangan = set_tokens & set_penjodoh_bilangan
    r_sepenjodoh_bilangan = set_tokens & set_sepenjodoh_bilangan
    r_reserved = set_tokens & set_reserved
    r = (t, tokens, tokens_lower, r_penjodoh_bilangan | r_sepenjodoh_bilangan)
    augment_4_0(t_, r)
    augment_4_1(t_, r)
    r = (t, tokens, tokens_lower, r_reserved)
    augment_4_2(t_, r)
    results.append((t, t_))

100%|██████████| 1/1 [00:00<00:00, 114.47it/s]


In [14]:
results

[([['berjuta-juta', 2], ['itu', 2]], [['berjuta-juta', 2], ['itu', 2]])]